<a href="https://colab.research.google.com/github/kaushalkuma-r/Machine-Learning-Course/blob/main/FINAL_ML_Assignment1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Import tools

In [ ]:
import numpy as np
import pandas as pd
import random
random.seed(42)

## Get the data

In [ ]:
visitors_df=pd.read_csv("https://raw.githubusercontent.com/kaushalkuma-r/Machine-Learning-Course/main/visitors-covid.csv")
covid_df=pd.read_csv("https://raw.githubusercontent.com/kaushalkuma-r/Machine-Learning-Course/main/covid-data.csv")

In [ ]:
covid_df['date']=pd.to_datetime(covid_df['date'])
visitors_df['Day']=pd.to_datetime(visitors_df['Day'])
covid_df=covid_df[['iso_code', 'continent', 'location', 'date','new_cases']]
covid_df=covid_df[covid_df['location']=='India']
visitors_df=visitors_df[visitors_df['Entity']=='India']
visitors_df.rename(columns={'Day':'date'},inplace=True)
covid_merged=pd.merge(covid_df, visitors_df, on='date')
final_df=covid_merged.drop(columns=['iso_code', 'continent', 'location', 'date','Entity','Code'])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:5047: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


In [ ]:
final_df

,new_cases,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,0.0,0.667,1.667,0.000,2.000,3.000,3.000
1,0.0,0.500,1.750,0.000,2.000,3.250,3.000
2,0.0,0.400,1.800,0.200,1.800,2.800,3.200
3,0.0,0.500,2.000,0.000,2.333,3.167,3.333
4,0.0,-0.143,1.714,0.714,1.429,3.571,0.143
...,...,...,...,...,...,...,...
466,173790.0,-61.714,-25.000,24.143,-49.143,-41.000,-45.429
467,165553.0,-61.286,-24.429,23.714,-48.714,-40.000,-44.571
468,152734.0,-61.143,-24.714,23.714,-49.000,-39.143,-44.286
469,127510.0,-60.143,-23.429,23.286,-48.286,-38.000,-43.429


In [ ]:
data = final_df
data.head(5)

,new_cases,retail_and_recreation,grocery_and_pharmacy,residential,transit_stations,parks,workplaces
0,0.0,0.667,1.667,0.000,2.000,3.000,3.000
1,0.0,0.500,1.750,0.000,2.000,3.250,3.000
2,0.0,0.400,1.800,0.200,1.800,2.800,3.200
3,0.0,0.500,2.000,0.000,2.333,3.167,3.333
4,0.0,-0.143,1.714,0.714,1.429,3.571,0.143


## Train-Test split

In [ ]:
shuffle_df = final_df.sample(frac=1)

# Defining a size for train set 
train_size = int(0.8 * len(final_df))

# Splitting dataset 
train_set = shuffle_df[:train_size]
test_set = shuffle_df[train_size:]

X_train , Y_train = train_set.drop('new_cases' , axis = 1).values , train_set['new_cases'].values.reshape(-1,1)
X_test , Y_test = test_set.drop('new_cases' , axis = 1).values , test_set['new_cases'].values.reshape(-1,1)

# print(X_train[:5])
# print(Y_train[:5])
# print(X_test[:5])
# print(Y_test[:5])

# Updated CART Decision Tree Model for Regression using Residual Sum of Squares

### Node class for CART

In [ ]:
class Tree_Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, min_rss=None, value=None):
        ''' constructor ''' 
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.min_rss = min_rss
        
        # for leaf node
        self.value = value

### Tree class for CART

In [ ]:
class CART_DTR():
    def __init__(self, min_samples_split=2, max_depth=2):
              
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
        
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            
            best_split = self.get_best_split(dataset, num_samples, num_features)
            left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
        
            right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
            return Tree_Node(best_split["feature_index"], best_split["threshold"], 
                        left_subtree, right_subtree, best_split["min_rss"])
    
        # compute the mean for node to return when predicting the data
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Tree_Node(value=leaf_value)

    def average(self,col):
      #Function to calculate the possible mean values of a Column
      l1=[]
      for i in range(len(col)-1):
        l1.append((col[i]+col[i+1])/2)
      return l1

    def get_best_split(self, dataset, num_samples, num_features):
              
        # dictionary to store the best split
        best_split = {}
        min_rss = +float("inf")
        # print(num_features)
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            # calling to get the mean of two consecutive elements of a feature.
            possible_thresholds_list=self.average(feature_values.tolist())
            possible_thresholds = np.array(possible_thresholds_list)
            # print(possible_thresholds)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
                # get current split
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    # compute  residual sum of square for both left and right datasets.
                    rss_left = self.rss(left_y)
                    rss_right=self.rss(right_y)
                    curr_rss=rss_left+rss_right
                    # update the best split to get the minimum rss out of each posssible thresholds for each feature column.
                    if curr_rss<min_rss:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["min_rss"] = curr_rss
                        min_rss = curr_rss
                        
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        # ''' function to split the data '''
        
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right

    def calculate_leaf_value(self, Y):
        # ''' function to compute leaf node '''
        val = np.mean(Y)
        return val
    
    def fit(self, X, Y):
        # ''' function to train the tree '''
        
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)
        
    def helper_predict(self, x, tree):
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.helper_predict(x, tree.left)
        else:
            return self.helper_predict(x, tree.right)
    
    def predict(self, X):
    # Function to make predictions        
        predictions = [self.helper_predict(x, self.root) for x in X]
        return predictions
    def rss(self,X):
      # Function to calculate the rss 
        mean_data=sum(X)
        rss=0
        for i in X:
          rss+=(mean_data-i)**2
        return rss

# ID3-A for Regression using Standard Deviation

In [ ]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, var_red=None, value=None):
        
        # for decision node
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.var_red = var_red
        
        # for leaf node
        self.value = value

In [ ]:
class ID3_A_Regressor():
    def __init__(self, min_samples_split=2, max_depth=2):
        
        # initialize the root of the tree 
        self.root = None
        
        # stopping conditions
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        
    def build_tree(self, dataset, curr_depth=0):
               
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        best_split = {}
        # split until stopping conditions are met
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            # find the best split
            best_split = self.get_best_split(dataset, num_samples, num_features)
            left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
            right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
            # creating the node for best split at this point in the tree
            return Node(best_split["feature_index"], best_split["threshold"], 
                        left_subtree, right_subtree, best_split["var_red"])
        
        # we are creating a leaf value for each Node, using the leaf_value we are creating our tree.
        leaf_value = self.calculate_leaf_value(Y)
        # return leaf node
        return Node(value=leaf_value)
    def average(self,col):
      #Function to calculate the possible mean values of a Column
      l1=[]
      for i in range(len(col)-1):
        l1.append((col[i]+col[i+1])/2)
      return l1

    def get_best_split(self, dataset, num_samples, num_features):
             
        # dictionary to store the best split
        best_split = {}
        max_var_red = -float("inf")
        # print(num_features)
        # loop over all the features
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds_list=self.average(feature_values.tolist())
            possible_thresholds = np.array(possible_thresholds_list)
            # print(possible_thres)
            # loop over all the feature values present in the data
            for threshold in possible_thresholds:
  
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                # check if childs are not null
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
  
                    curr_var_red = self.var_reduction(y, left_y, right_y)
                    if curr_var_red>max_var_red:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["var_red"] = curr_var_red
                        max_var_red = curr_var_red
        return best_split
    
    def split(self, dataset, feature_index, threshold):       
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def var_reduction(self, parent, left, right):
        
        reduction = np.var(parent) - ((len(left) / len(parent)) * np.var(left) + (len(right) / len(parent)) * np.var(right))
        return reduction
    
    def calculate_leaf_value(self, Y):
        return  np.mean(Y)
    def fit(self, X, Y):
                
        dataset = np.concatenate((X, Y), axis=1)
        self.root = self.build_tree(dataset)

    def make_prediction(self, x, tree):
            
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)
    
    def predict(self, X):      
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions

## Fitting the model for CART

In [ ]:
cart_regressor = CART_DTR(min_samples_split=2, max_depth=7)
cart_regressor.fit(X_train,Y_train)

### Fitting the model for ID3-A

In [ ]:
id3_regressor = ID3_A_Regressor(min_samples_split=2, max_depth=4)
id3_regressor.fit(X_train,Y_train)

## Testing the created models and calculating their r2score.

In [ ]:
Y_pred_id3 = id3_regressor.predict(X_test) 
Y_pred_cart = cart_regressor.predict(X_test) 

In [ ]:
Y_test=[i[0] for i in Y_test.tolist()]

In [ ]:
Y_test=np.array(Y_test)
Y_pred_id3=np.array(Y_pred_id3)

In [ ]:
Y_pred_cart=np.array(Y_pred_cart)

In [ ]:
def r2Score(y_pred,y_actual):
    sum_square_pred = np.sum((y_pred-y_actual)**2)
    sum_squares_actual = np.sum((y_actual- y_actual.mean())**2)
    score = (1- sum_square_pred/sum_squares_actual)
    return score

In [ ]:
score_id3=r2Score(Y_pred_id3, Y_test)
score_id3

0.946221976112327

In [ ]:
score_cart=r2Score(Y_pred_cart, Y_test)
score_cart

0.9753145431319843

In [ ]:
def rmse(prediction, target):
    return np.sqrt(((prediction- target) ** 2).mean())

# Tuning the Hyperparameters for obtaining the best model for the current dataset.

In [ ]:
min_samples_split=[5,10,15,20,25]
max_depth=[4,5,6,7,8,9,10,11,12]
dir=[]

In [ ]:
for i in min_samples_split:
  for j in max_depth:
    l1={}
    l1['min_samples_split']=i
    l1['max_depth']=j
    cart_regressor = CART_DTR(min_samples_split=i, max_depth=j)
    cart_regressor.fit(X_train,Y_train)
    id3_regressor = ID3_A_Regressor(min_samples_split=i, max_depth=j)
    id3_regressor.fit(X_train,Y_train)
    Y_pred_id3 = id3_regressor.predict(X_test) 
    Y_pred_cart = cart_regressor.predict(X_test) 
    Y_pred_cart=np.array(Y_pred_cart)
    Y_pred_id3=np.array(Y_pred_id3)
    score_id3=r2Score(Y_pred_id3, Y_test)
    score_cart=r2Score(Y_pred_cart, Y_test)
    # calculate the rmse for both ID3A and Cart predictions
    rm_score_id3=rmse(Y_pred_id3,Y_test)
    rm_score_cart=rmse(Y_pred_cart,Y_test)
    l1['ID3-A-r2Score']=score_id3
    l1['CART-r2Score']=score_cart
    l1['ID3-A-rmse']=rm_score_id3
    l1['CART-rmse']=rm_score_cart
    dir.append(l1)

In [ ]:
df=pd.DataFrame(dir)

In [ ]:
df

,min_samples_split,max_depth,ID3-A-r2Score,CART-r2Score,ID3-A-rmse,CART-rmse
0,5,4,0.946222,0.871868,15475.471263,23887.502412
1,5,5,0.960985,0.937066,13181.283673,16741.106670
2,5,6,0.961865,0.940014,13031.760133,16344.285521
3,5,7,0.965520,0.941005,12391.447187,16208.687901
4,5,8,0.965546,0.941008,12386.787773,16208.364532
5,5,9,0.965332,0.941007,12425.223671,16208.453250
6,5,10,0.965301,0.941007,12430.800438,16208.453266
7,5,11,0.965356,0.941007,12421.001000,16208.453273
8,5,12,0.965356,0.941007,12421.001000,16208.453277
9,10,4,0.949797,0.850612,14952.218983,25792.834922


In [ ]:
df1=df[df['ID3-A-r2Score'] == df['ID3-A-r2Score'].max()]

In [ ]:
p=df1['max_depth'].tolist()
p1=df1['min_samples_split'].tolist()


In [ ]:
p1

[10]

In [ ]:
df2=df[df['CART-r2Score'] == df['CART-r2Score'].max()]

In [ ]:
p2=df2['max_depth'].tolist()
p3=df2['min_samples_split'].tolist()

In [ ]:
df1

,min_samples_split,max_depth,ID3-A-r2Score,CART-r2Score,ID3-A-rmse,CART-rmse
14,10,9,0.969748,0.857349,11606.983549,25204.546845


The above code works as hyperparameter tuning and finds the best hyperparameter for the model. We take r2 score as reference and note the value of min_sample_split and max_depth at which we have maximum value of it.

In [ ]:
cart_regressor = CART_DTR(min_samples_split=p3[0], max_depth=p2[0])
cart_regressor.fit(X_train,Y_train)

In [ ]:
id3_regressor = ID3_A_Regressor(min_samples_split=p1[0], max_depth=p[0])
id3_regressor.fit(X_train,Y_train)

In [ ]:
Y_pred_id3 = id3_regressor.predict(X_test) 
Y_pred_cart = cart_regressor.predict(X_test) 
Y_pred_cart=np.array(Y_pred_cart)
Y_pred_id3=np.array(Y_pred_id3)
score_id3=r2Score(Y_pred_id3, Y_test)
score_cart=r2Score(Y_pred_cart, Y_test)
# calculate the rmse for both ID3A and Cart predictions
rm_score_id3=rmse(Y_pred_id3,Y_test)
rm_score_cart=rmse(Y_pred_cart,Y_test)

In [ ]:

score_id3,rm_score_id3

(0.969747880220882, 11606.983549112041)

In [ ]:
score_cart

0.9410076815603954

# K-Means Clustering for classifying the dataset

In [ ]:
shuffle_df = final_df.sample(frac=1,random_state=60)

# Defining a size for train set 
train_size = int(0.8 * len(final_df))

# Splitting dataset 
train_set = shuffle_df[:train_size]
test_set = shuffle_df[train_size:]

X_train , Y_train = train_set.drop('new_cases' , axis = 1).values , train_set['new_cases'].values.reshape(-1,1)
X_test , Y_test = test_set.drop('new_cases' , axis = 1).values , test_set['new_cases'].values.reshape(-1,1)

# print(X_train[:5])
# print(Y_train[:5])
# print(X_test[:5])
# print(Y_test[:5])

In [ ]:
# Select any 3 random points and Make clusters on them
random_indices = np.random.choice(X_train.shape[0], size=5, replace=False)
centroids = X_train[random_indices, :]
print(centroids)

[[-26.571   9.571  11.714  -7.857 -13.571 -18.   ]
 [-34.429  10.857  15.143 -20.714 -23.714 -26.714]
 [-20.286  20.429   7.714  -5.143  -9.571 -13.714]
 [-24.     15.571   6.857  -8.571 -16.    -14.714]
 [-53.857 -12.143  21.857 -39.714 -39.286 -41.429]]


In [ ]:
class Cluster_ID3():
        
    def __init__(self, k=5, max_iterations = 500):
        self.k = k
        self.max_iterations = max_iterations
        self.clusters_train = {}
        self.clusters_label = {}
        self.root = {}
    def euclidean_distance(self, point1, point2):
        return np.linalg.norm(point1-point2, axis=0)

    def train(self):
        for i in range(self.k):
            temp = ID3_A_Regressor(min_samples_split=5,max_depth=6)
            temp.fit(X = self.clusters_train[i] , Y = self.clusters_label[i])
            self.root[i] = temp

    def make_clusters(self , X , y, centroids):
        # Set the range for value of k (number of centroids)
        for i in range(self.k):
            self.clusters_train[i] = []
            self.clusters_label[i] = []
        for i in range(len(X)):
            euc_dist = []
            for j in range(self.k):
                euc_dist.append(np.linalg.norm(X[i] - centroids[j]))
            # Append the cluster of data to the dictionary
            self.clusters_train[euc_dist.index(min(euc_dist))].append(X[i])
            self.clusters_label[euc_dist.index(min(euc_dist))].append(y[i])

    def predict(self, X):
        y_pred = []
        for i in range(len(X)):
            euc_dist = []
            for j in range(self.k):
                euc_dist.append(np.linalg.norm(X[i] - centroids[j]))
            idx = euc_dist.index(min(euc_dist))
            # print(idx)
            l = []
            l.append(X[i])
            y_pred.append(self.root[idx].predict(l))

        return y_pred
      
    def fit(self, data):
        self.centroids = {}
        for i in range(self.k):
            self.centroids[i] = data[i]
  
        for i in range(self.max_iterations):
            self.classes = {}
            for j in range(self.k):
                self.classes[j] = []
            for point in data:
                distances = []
                for index in self.centroids:
                    distances.append(self.euclidean_distance(point,self.centroids[index]))
                

                cluster_index = distances.index(min(distances))
                self.classes[cluster_index].append(point)
      
            for cluster_index in self.classes:
                self.centroids[cluster_index] = np.average(self.classes[cluster_index], axis = 0)

In [ ]:
model = Cluster_ID3()
model.fit(X_train)
model.make_clusters(X_train , Y_train , model.centroids)
model.train()

In [ ]:
print(model.clusters_train)

{0: [array([-0.429,  1.   ,  0.   ,  0.714,  1.857,  4.571]), array([-1.   , -0.714,  0.143,  0.714,  2.429,  2.429]), array([-8.857, -3.429,  3.286, -6.571, -2.429, -8.143]), array([-0.286,  3.429,  0.286,  1.857,  1.857,  5.143]), array([-7.714, -0.143,  2.143, -4.714, -2.143, -3.   ]), array([-0.429,  0.429,  0.   ,  0.857,  1.857,  4.143]), array([-1.   , -0.571,  0.   ,  1.   ,  1.714,  3.857]), array([-3.571,  0.286,  2.286, -2.286,  0.714, -5.429]), array([-1.571, -0.857,  0.857,  0.   ,  2.857, -1.   ]), array([-8.714,  0.143,  2.571, -6.   , -2.714, -3.429]), array([-0.143,  4.143,  0.143,  2.286,  2.   ,  5.571]), array([-10.571,   0.143,   3.286,  -8.   ,  -4.   ,  -5.   ]), array([0.667, 1.667, 0.   , 2.   , 3.   , 3.   ]), array([0.5 , 1.75, 0.  , 2.  , 3.25, 3.  ]), array([-0.571,  0.143,  0.   ,  1.   ,  1.714,  4.   ]), array([-4.429, -0.143,  2.286, -3.286,  0.571, -6.857]), array([-0.143,  1.714,  0.714,  1.429,  3.571,  0.143]), array([-0.571,  2.429,  0.286,  1.143,

# Predicting new_cases using ID3-A after clustering the datapoints.

In [ ]:
Y_pred_K_ID3 = model.predict(X_test)
print(Y_pred_K_ID3)

[[62905.0], [7933.0], [36892.5], [33690.666666666664], [6198.0], [33690.666666666664], [196427.0], [8442.0], [192555.5], [38.0], [391263.0], [87948.0], [7933.0], [36892.5], [28347.0], [70.0], [6198.0], [36892.5], [54773.0], [346658.0], [36892.5], [14348.25], [378907.75], [8442.0], [54773.0], [33690.666666666664], [33690.666666666664], [28347.0], [196427.0], [33690.666666666664], [23294.333333333332], [38.0], [38.0], [183657.0], [6198.0], [6198.0], [37740.0], [38.0], [9453.5], [6198.0], [6198.0], [11033.0], [6198.0], [38.0], [33690.666666666664], [6198.0], [36892.5], [6198.0], [36892.5], [14348.25], [70.0], [17563.8], [6198.0], [38.0], [192555.5], [38.0], [38.0], [192555.5], [38.0], [38.0], [28347.0], [192555.5], [192555.5], [14959.5], [378907.75], [6198.0], [346658.0], [70.0], [14348.25], [37740.0], [346658.0], [70.0], [68206.5], [192555.5], [14348.25], [378907.75], [38.0], [38.0], [6198.0], [11033.0], [6198.0], [54746.75], [33690.666666666664], [346658.0], [48507.6], [54773.0], [36892

In [ ]:
Y_pred_K_ID3=[i[0] for i in Y_pred_K_ID3]

In [ ]:
Y_test=[i[0] for i in Y_test]

In [ ]:
r2score_k_id3=r2Score(np.array(Y_pred_K_ID3), np.array(Y_test))
r2score_k_id3

0.46197827044863027

In [ ]:
class Cluster_CART():
        
    def __init__(self, k=5, max_iterations = 500):
        self.k = k
        self.max_iterations = max_iterations
        self.clusters_train = {}
        self.clusters_label = {}
        self.root = {}
    def euclidean_distance(self, point1, point2):
        return np.linalg.norm(point1-point2, axis=0)

    def train(self):
        for i in range(self.k):
            temp = CART_DTR()
            temp.fit(X = self.clusters_train[i] , Y = self.clusters_label[i])
            self.root[i] = temp

    def make_clusters(self , X , y, centroids):
        # Set the range for value of k (number of centroids)
        for i in range(self.k):
            self.clusters_train[i] = []
            self.clusters_label[i] = []
        for i in range(len(X)):
            euc_dist = []
            for j in range(self.k):
                euc_dist.append(np.linalg.norm(X[i] - centroids[j]))
            # Append the cluster of data to the dictionary
            self.clusters_train[euc_dist.index(min(euc_dist))].append(X[i])
            self.clusters_label[euc_dist.index(min(euc_dist))].append(y[i])

    def predict(self, X):
        y_pred = []
        for i in range(len(X)):
            euc_dist = []
            for j in range(self.k):
                euc_dist.append(np.linalg.norm(X[i] - centroids[j]))
            idx = euc_dist.index(min(euc_dist))
            # print(idx)
            l = []
            l.append(X[i])
            y_pred.append(self.root[idx].predict(l))

        return y_pred
      
    def fit(self, data):
        self.centroids = {}
        for i in range(self.k):
            self.centroids[i] = data[i]
  
        for i in range(self.max_iterations):
            self.classes = {}
            for j in range(self.k):
                self.classes[j] = []
            for point in data:
                distances = []
                for index in self.centroids:
                    distances.append(self.euclidean_distance(point,self.centroids[index]))
                

                cluster_index = distances.index(min(distances))
                self.classes[cluster_index].append(point)
      
            for cluster_index in self.classes:
                self.centroids[cluster_index] = np.average(self.classes[cluster_index], axis = 0)

In [ ]:
model = Cluster_CART()
model.fit(X_train)
model.make_clusters(X_train , Y_train , model.centroids)
model.train()

# Predicting new_cases using updated CART model after clustering the datapoints.

In [ ]:
Y_pred_K_Cart = model.predict(X_test)
print(Y_pred_K_Cart)

[[86142.0], [36297.086956521736], [42964.0], [27897.923076923078], [36297.086956521736], [27897.923076923078], [354374.3333333333], [21438.342857142856], [18731.625], [38.0], [361322.1666666667], [27897.923076923078], [36297.086956521736], [42964.0], [21438.342857142856], [107623.33333333333], [36297.086956521736], [42964.0], [21438.342857142856], [361322.1666666667], [42964.0], [18731.625], [354374.3333333333], [21438.342857142856], [36297.086956521736], [18731.625], [27897.923076923078], [21438.342857142856], [354374.3333333333], [27897.923076923078], [21438.342857142856], [38.0], [38.0], [127535.33333333333], [36297.086956521736], [36297.086956521736], [36297.086956521736], [38.0], [21438.342857142856], [36297.086956521736], [36297.086956521736], [21438.342857142856], [36297.086956521736], [38.0], [18731.625], [36297.086956521736], [42964.0], [36297.086956521736], [42964.0], [18731.625], [107623.33333333333], [21438.342857142856], [36297.086956521736], [38.0], [159206.2], [38.0], [3

In [ ]:
Y_pred_K_Cart=[i[0] for i in Y_pred_K_Cart]

In [ ]:
r2score_k_cart=r2Score(np.array(Y_pred_K_Cart), np.array(Y_test))
r2score_k_cart

0.20663881514930704